# 1. Coleta de Dados

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq

In [2]:
campeonatos = []
for i in range(2007,2017):
    campeonatos.append(i)
campeonatos

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [6]:
urls = []
for i in range(len(campeonatos)):
    urls.append('https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id='
                +str(campeonatos[i]))
urls

['https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2007',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2008',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2009',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2010',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2011',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2012',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2013',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2014',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id

In [141]:
allNames = []
allSquads = []
allAges = []
allFPs = []
allValues = []
allAvgValues = []

allPos = []
allGDs = []
allPoints = []

headers = {'User-Agent': 'ajgan'}

for a in range(len(urls)):
    r = rq.get(urls[a], headers=headers)
    parsed_page = BeautifulSoup(r.text,"html.parser")
    
    teams = parsed_page.find_all('tr')[11:31]
    positions = parsed_page.find_all('tr')[72:92]
    
    teamName = []
    teamSquad = []
    teamAge = []
    teamFP = []
    teamValue = []
    teamAvgValue = []

    teamPos = []
    teamGD = []
    teamPoints = []

    for i in range(len(teams)):
        teamName.append(teams[i].find_all('td')[2].text)
        teamSquad.append(teams[i].find_all('td')[3].text)
        teamAge.append(teams[i].find_all('td')[4].text)
        teamFP.append(teams[i].find_all('td')[5].text)
        teamValue.append(teams[i].find_all('td')[6].text)
        teamAvgValue.append(teams[i].find_all('td')[7].text)
    for j in range(len(teamName)):
        for k in range(len(positions)):
            if (teamName[j] == positions[k].find_all('a')[1].text):
                teamPos.append(positions[k].find_all('td')[3].text)
                teamGD.append(positions[k].find_all('td')[4].text)
                teamPoints.append(positions[k].find_all('td')[5].text)
                break
    
    for m in range(len(teamName)):
        teamName[m] = teamName[m] + " - " + str(campeonatos[a]+1)
        
    allNames.extend(teamName)
    allSquads.extend(teamSquad)
    allAges.extend(teamAge)
    allFPs.extend(teamFP)
    allValues.extend(teamValue)
    allAvgValues.extend(teamAvgValue)

    allPos.extend(teamPos)
    allGDs.extend(teamGD)
    allPoints.extend(teamPoints)

In [143]:
columns = ['Equipe', 'Pos', 'Pts', 'SG', 'TamanhoElenco', 'IdadeElenco', 'JogadoresEstrangeiros','Valor', 'ValorMedio']
brasileirao = pd.DataFrame(list(zip(allNames, allPos, allPoints, allGDs, allSquads, allAges, allFPs, allValues, allAvgValues))
                          , columns=columns)

In [144]:
brasileirao

,Equipe,Pos,Pts,SG,TamanhoElenco,IdadeElenco,JogadoresEstrangeiros,Valor,ValorMedio
0,São Paulo - 2008,38,75,30,41,"24,2",1,"54,15 Mill. €","1,32 Mill. €"
1,Palmeiras - 2008,38,65,10,28,"23,4",1,"23,60 Mill. €",843 Th. €
2,Flamengo - 2008,38,64,20,30,"23,5",1,"18,50 Mill. €",617 Th. €
3,Internacional - 2008,38,54,1,34,"23,3",4,"18,00 Mill. €",529 Th. €
4,Grêmio - 2008,38,72,24,34,"22,7",1,"11,90 Mill. €",350 Th. €
5,Fluminense - 2008,38,45,1,35,"23,2",1,"8,70 Mill. €",249 Th. €
6,Portuguesa - 2008,38,38,-22,33,"24,8",1,"6,15 Mill. €",186 Th. €
7,Santos FC - 2008,38,45,-9,29,"22,4",3,"6,05 Mill. €",209 Th. €
8,Atlético-MG - 2008,38,48,-10,42,"23,4",4,"5,54 Mill. €",132 Th. €
9,Cruzeiro - 2008,38,67,15,25,"23,1",1,"4,95 Mill. €",198 Th. €


In [145]:
brasileirao.to_csv("brasileirao.csv")